# Advanced Analytics: Machine Learning & User Segmentation

## Objective
Implement advanced analytics including skip prediction modeling, user segmentation through clustering, and retention analysis to drive data-driven product decisions.

## 1. Setup and Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('../data/spotify_cleaned.csv')
session_stats = pd.read_csv('../data/session_statistics.csv')

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Loaded {len(df):,} records for advanced analytics")
print(f"Target variable (skip rate): {df['is_skip'].mean():.1%}")

## 2. Skip Prediction Model Development

In [ ]:
# Feature engineering for skip prediction
def create_skip_features(df):
    features_df = df.copy()
    
    # Temporal features
    features_df['is_weekend'] = features_df['day_of_week_num'].isin([5, 6]).astype(int)
    features_df['is_peak_hour'] = features_df['hour_of_day'].isin([17, 18, 19, 20]).astype(int)
    features_df['is_night'] = features_df['hour_of_day'].isin([22, 23, 0, 1, 2, 3, 4, 5]).astype(int)
    
    # Track features
    features_df['track_length_seconds'] = features_df['estimated_track_length_ms'] / 1000
    features_df['is_long_track'] = (features_df['track_length_seconds'] > 240).astype(int)
    features_df['is_short_track'] = (features_df['track_length_seconds'] < 120).astype(int)
    
    # Behavioral features
    features_df['shuffle_int'] = features_df['shuffle'].astype(int)
    
    # Platform encoding
    platform_dummies = pd.get_dummies(features_df['platform'], prefix='platform')
    features_df = pd.concat([features_df, platform_dummies], axis=1)
    
    # Reason start encoding (top reasons only)
    top_reasons = features_df['reason_start'].value_counts().head(5).index
    for reason in top_reasons:
        features_df[f'reason_start_{reason}'] = (features_df['reason_start'] == reason).astype(int)
    
    return features_df

# Create features
ml_df = create_skip_features(df)

# Select features for modeling
feature_columns = [
    'hour_of_day', 'day_of_week_num', 'is_weekend', 'is_peak_hour', 'is_night',
    'track_length_seconds', 'is_long_track', 'is_short_track', 'shuffle_int',
    'platform_Android', 'platform_iOS', 'platform_web player',
    'reason_start_clickrow', 'reason_start_trackdone', 'reason_start_autoplay'
]

# Filter features that exist
available_features = [col for col in feature_columns if col in ml_df.columns]
print(f"Using {len(available_features)} features for modeling: {available_features}")

X = ml_df[available_features].fillna(0)
y = ml_df['is_skip'].astype(int)

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target distribution: Skip={y.sum():,} ({y.mean():.1%}), No Skip={len(y)-y.sum():,} ({1-y.mean():.1%})")

In [ ]:
# Split data and train models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train multiple models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=6)
}

model_results = {}

print("🤖 SKIP PREDICTION MODEL TRAINING")
print("=" * 40)

for name, model in models.items():
    # Train model
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = (y_pred == y_test).mean()
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'auc': auc_score,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"{name}:")
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  AUC Score: {auc_score:.3f}")
    print()

# Select best model
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['auc'])
best_model = model_results[best_model_name]
print(f"🏆 Best model: {best_model_name} (AUC: {best_model['auc']:.3f})")

In [ ]:
# Detailed evaluation of best model
print(f"📊 DETAILED EVALUATION: {best_model_name}")
print("=" * 45)

# Classification report
print("Classification Report:")
print(classification_report(y_test, best_model['predictions']))

# Feature importance (if available)
if hasattr(best_model['model'], 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': available_features,
        'importance': best_model['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Feature Importances:")
    print(feature_importance.head(10))
    
    # Visualize feature importance
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
    plt.title(f'Top 10 Feature Importances - {best_model_name}')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()

# ROC Curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, best_model['probabilities'])
plt.plot(fpr, tpr, linewidth=2, label=f'{best_model_name} (AUC = {best_model["auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Skip Prediction')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, best_model['predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Skip', 'Skip'], yticklabels=['No Skip', 'Skip'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 3. User Segmentation through Clustering

In [ ]:
# Create user-level features for clustering
user_features = df.groupby('spotify_track_uri').agg({
    'seconds_played': ['count', 'sum', 'mean'],
    'is_skip': 'mean',
    'percent_played': 'mean',
    'shuffle': 'mean',
    'hour_of_day': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.mean(),
    'platform': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
    'day_of_week_num': 'mean'
}).round(3)

user_features.columns = ['play_count', 'total_seconds', 'avg_seconds_per_play', 
                        'skip_rate', 'avg_percent_played', 'shuffle_rate',
                        'preferred_hour', 'preferred_platform', 'avg_day_of_week']

# Add more behavioral features
user_features['total_minutes'] = user_features['total_seconds'] / 60
user_features['engagement_score'] = (
    user_features['avg_percent_played'] * (1 - user_features['skip_rate'])
)

# Filter for tracks with sufficient data
user_features = user_features[user_features['play_count'] >= 3]

print(f"Created user features for {len(user_features):,} tracks/users")
print(f"\nUser features summary:")
print(user_features.describe())

In [ ]:
# Prepare clustering features
clustering_features = [
    'play_count', 'avg_seconds_per_play', 'skip_rate', 
    'avg_percent_played', 'shuffle_rate', 'engagement_score'
]

X_cluster = user_features[clustering_features].fillna(0)

# Scale features for clustering
cluster_scaler = StandardScaler()
X_cluster_scaled = cluster_scaler.fit_transform(X_cluster)

# Find optimal number of clusters
k_range = range(2, 10)
inertias = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_cluster_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_cluster_scaled, cluster_labels))

# Plot elbow curve and silhouette scores
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(k_range, inertias, marker='o')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('Elbow Method for Optimal k')
ax1.grid(True, alpha=0.3)

ax2.plot(k_range, silhouette_scores, marker='o', color='orange')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score for Different k')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Select optimal k (highest silhouette score)
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k} (Silhouette Score: {max(silhouette_scores):.3f})")

In [ ]:
# Perform final clustering
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
user_features['cluster'] = final_kmeans.fit_predict(X_cluster_scaled)

# Analyze clusters
cluster_analysis = user_features.groupby('cluster').agg({
    'play_count': ['count', 'mean'],
    'avg_seconds_per_play': 'mean',
    'skip_rate': 'mean',
    'avg_percent_played': 'mean',
    'shuffle_rate': 'mean',
    'engagement_score': 'mean',
    'total_minutes': 'mean'
}).round(3)

cluster_analysis.columns = ['cluster_size', 'avg_play_count', 'avg_seconds_per_play',
                           'avg_skip_rate', 'avg_percent_played', 'avg_shuffle_rate',
                           'avg_engagement_score', 'avg_total_minutes']

print("🎯 USER SEGMENTATION ANALYSIS")
print("=" * 35)
print(cluster_analysis)

# Create cluster profiles
cluster_profiles = {
    0: "Casual Listeners",
    1: "Engaged Users",
    2: "Power Users",
    3: "Exploratory Users",
    4: "Skip-Heavy Users"
}

# Assign profiles based on characteristics
print(f"\n👥 CLUSTER PROFILES:")
for cluster_id in range(optimal_k):
    stats = cluster_analysis.loc[cluster_id]
    size = stats['cluster_size']
    skip_rate = stats['avg_skip_rate']
    engagement = stats['avg_engagement_score']
    play_count = stats['avg_play_count']
    
    print(f"\nCluster {cluster_id} ({size:,} users, {size/len(user_features)*100:.1f}%):")
    print(f"  Average plays: {play_count:.1f}")
    print(f"  Skip rate: {skip_rate:.1%}")
    print(f"  Engagement score: {engagement:.2f}")
    print(f"  Total listening: {stats['avg_total_minutes']:.1f} minutes")
    
    # Characterize cluster
    if skip_rate > 0.5:
        profile = "Skip-Heavy Users (Low Satisfaction)"
    elif engagement > 50 and play_count > 10:
        profile = "Power Users (High Value)"
    elif engagement > 30:
        profile = "Engaged Users (Core Audience)"
    else:
        profile = "Casual Listeners (Growth Opportunity)"
    
    print(f"  Profile: {profile}")

In [ ]:
# Visualize clusters
# PCA for 2D visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_cluster_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=user_features['cluster'], 
                     cmap='viridis', alpha=0.6, s=50)
plt.colorbar(scatter)
plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('User Segmentation Clusters (PCA Visualization)')

# Add cluster centers
centers_pca = pca.transform(final_kmeans.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], 
           c='red', marker='x', s=200, linewidths=3, label='Centroids')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Feature importance in clusters
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feature in enumerate(clustering_features):
    cluster_feature_means = user_features.groupby('cluster')[feature].mean()
    axes[i].bar(range(optimal_k), cluster_feature_means.values, 
               color=plt.cm.viridis(np.linspace(0, 1, optimal_k)))
    axes[i].set_title(f'{feature.replace("_", " ").title()}')
    axes[i].set_xlabel('Cluster')
    axes[i].set_xticks(range(optimal_k))

plt.tight_layout()
plt.show()

## 4. Retention and Churn Analysis

In [ ]:
# Daily activity analysis for retention
df['date'] = df['timestamp'].dt.date
daily_activity = df.groupby(['date']).agg({
    'spotify_track_uri': 'count',
    'seconds_played': 'sum',
    'is_skip': 'mean'
}).round(3)

daily_activity.columns = ['daily_plays', 'daily_seconds', 'daily_skip_rate']
daily_activity['daily_minutes'] = daily_activity['daily_seconds'] / 60

# Convert date back to datetime for analysis
daily_activity.reset_index(inplace=True)
daily_activity['date'] = pd.to_datetime(daily_activity['date'])
daily_activity = daily_activity.sort_values('date')

# Calculate retention metrics
total_days = (daily_activity['date'].max() - daily_activity['date'].min()).days + 1
active_days = len(daily_activity)
retention_rate = active_days / total_days

print("📈 RETENTION & CHURN ANALYSIS")
print("=" * 35)
print(f"Total period: {total_days} days")
print(f"Active days: {active_days} days")
print(f"Daily retention rate: {retention_rate:.1%}")
print(f"Average daily plays: {daily_activity['daily_plays'].mean():.0f}")
print(f"Average daily listening: {daily_activity['daily_minutes'].mean():.1f} minutes")

# Activity consistency (engagement stability)
activity_cv = daily_activity['daily_plays'].std() / daily_activity['daily_plays'].mean()
print(f"Activity consistency (lower is better): {activity_cv:.2f}")

# Weekly retention patterns
daily_activity['week'] = daily_activity['date'].dt.isocalendar().week
weekly_activity = daily_activity.groupby('week').agg({
    'daily_plays': ['count', 'sum', 'mean'],
    'daily_minutes': ['sum', 'mean'],
    'daily_skip_rate': 'mean'
}).round(2)

weekly_activity.columns = ['active_days_per_week', 'total_weekly_plays', 'avg_daily_plays',
                          'total_weekly_minutes', 'avg_daily_minutes', 'avg_weekly_skip_rate']

print(f"\nWeekly patterns:")
print(f"Average active days per week: {weekly_activity['active_days_per_week'].mean():.1f}")
print(f"Most active week: {weekly_activity['total_weekly_plays'].max():,.0f} plays")
print(f"Least active week: {weekly_activity['total_weekly_plays'].min():,.0f} plays")

In [ ]:
# Churn risk identification
# Define churn risk based on recent activity decline
daily_activity = daily_activity.sort_values('date')
daily_activity['rolling_7d_plays'] = daily_activity['daily_plays'].rolling(7, center=True).mean()
daily_activity['rolling_7d_minutes'] = daily_activity['daily_minutes'].rolling(7, center=True).mean()

# Activity trend (last 7 days vs previous 7 days)
recent_activity = daily_activity.tail(7)['daily_plays'].mean()
previous_activity = daily_activity.iloc[-14:-7]['daily_plays'].mean() if len(daily_activity) >= 14 else recent_activity

activity_trend = (recent_activity - previous_activity) / previous_activity if previous_activity > 0 else 0

print(f"\n🔄 ACTIVITY TRENDS:")
print(f"Recent 7-day avg: {recent_activity:.1f} plays/day")
print(f"Previous 7-day avg: {previous_activity:.1f} plays/day")
print(f"Activity trend: {activity_trend:+.1%}")

if activity_trend < -0.2:
    churn_risk = "High"
elif activity_trend < -0.1:
    churn_risk = "Medium"
else:
    churn_risk = "Low"

print(f"Churn risk level: {churn_risk}")

# Visualize retention trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Daily activity trend
ax1.plot(daily_activity['date'], daily_activity['daily_plays'], alpha=0.5, label='Daily')
ax1.plot(daily_activity['date'], daily_activity['rolling_7d_plays'], 
         linewidth=2, color='red', label='7-day Average')
ax1.set_title('Daily Activity Trend')
ax1.set_ylabel('Plays per Day')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Weekly activity pattern
ax2.bar(weekly_activity.index, weekly_activity['total_weekly_plays'], 
        color='skyblue', alpha=0.7)
ax2.set_title('Weekly Activity Pattern')
ax2.set_xlabel('Week Number')
ax2.set_ylabel('Total Weekly Plays')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Advanced Business Insights & Recommendations

In [ ]:
# Synthesize insights from ML models
print("🧠 ADVANCED ANALYTICS INSIGHTS")
print("=" * 40)

# Skip prediction insights
if hasattr(best_model['model'], 'feature_importances_'):
    top_skip_drivers = feature_importance.head(3)
    print(f"\n🎯 SKIP PREDICTION INSIGHTS:")
    print(f"• Model accuracy: {best_model['accuracy']:.1%}")
    print(f"• Top skip drivers:")
    for _, row in top_skip_drivers.iterrows():
        print(f"  - {row['feature']}: {row['importance']:.3f} importance")

# User segmentation insights
print(f"\n👥 USER SEGMENTATION INSIGHTS:")
high_value_clusters = cluster_analysis[cluster_analysis['avg_engagement_score'] > 
                                     cluster_analysis['avg_engagement_score'].mean()]
risk_clusters = cluster_analysis[cluster_analysis['avg_skip_rate'] > 0.5]

print(f"• Identified {optimal_k} distinct user segments")
print(f"• {len(high_value_clusters)} high-engagement segments ({len(high_value_clusters)/optimal_k*100:.0f}%)")
if len(risk_clusters) > 0:
    print(f"• {len(risk_clusters)} at-risk segments with >50% skip rate")

# Retention insights
print(f"\n📈 RETENTION INSIGHTS:")
print(f"• Daily retention rate: {retention_rate:.1%}")
print(f"• Activity trend: {activity_trend:+.1%}")
print(f"• Churn risk level: {churn_risk}")

# Action recommendations
print(f"\n🎯 ML-DRIVEN RECOMMENDATIONS:")

recommendations = []

# Skip prediction recommendations
if 'track_length_seconds' in feature_importance.head(3)['feature'].values:
    recommendations.append("Optimize track length recommendations - length is a key skip driver")

if 'shuffle_int' in feature_importance.head(3)['feature'].values:
    recommendations.append("Improve shuffle algorithm - shuffle mode affects skip behavior")

# User segmentation recommendations
total_users = cluster_analysis['cluster_size'].sum()
casual_users_pct = (cluster_analysis[cluster_analysis['avg_engagement_score'] < 30]['cluster_size'].sum() / total_users) * 100

if casual_users_pct > 40:
    recommendations.append(f"Focus on converting {casual_users_pct:.0f}% casual users to engaged users")

if len(risk_clusters) > 0:
    risk_users_pct = (risk_clusters['cluster_size'].sum() / total_users) * 100
    recommendations.append(f"Implement retention strategy for {risk_users_pct:.0f}% high-skip users")

# Retention recommendations
if activity_trend < -0.1:
    recommendations.append("Implement re-engagement campaign - activity is declining")

if retention_rate < 0.7:
    recommendations.append("Improve daily retention through personalization and notifications")

# Platform-specific recommendations
platform_performance = df.groupby('platform')['is_skip'].mean()
worst_platform = platform_performance.idxmax()
recommendations.append(f"Optimize {worst_platform} experience - highest skip rate platform")

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Save ML results
user_features.to_csv('../data/user_segments.csv')
daily_activity.to_csv('../data/retention_analysis.csv', index=False)

print(f"\n✅ Advanced analytics results saved to ../data/")

## 6. Model Performance Summary

In [ ]:
# Create comprehensive model summary
model_summary = pd.DataFrame({
    'Model': list(model_results.keys()),
    'Accuracy': [results['accuracy'] for results in model_results.values()],
    'AUC Score': [results['auc'] for results in model_results.values()]
})

print("🤖 MODEL PERFORMANCE SUMMARY")
print("=" * 35)
print(model_summary.round(3))

# Clustering summary
clustering_summary = {
    'Optimal Clusters': optimal_k,
    'Silhouette Score': max(silhouette_scores),
    'Total Users Segmented': len(user_features),
    'Variance Explained (PCA)': sum(pca.explained_variance_ratio_)
}

print(f"\n🎯 CLUSTERING SUMMARY:")
for metric, value in clustering_summary.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value:,}")

# Export final summary
model_summary.to_csv('../reports/model_performance.csv', index=False)

final_summary = {
    'Best Model': best_model_name,
    'Best Model AUC': best_model['auc'],
    'User Segments': optimal_k,
    'Retention Rate': retention_rate,
    'Activity Trend': activity_trend,
    'Churn Risk': churn_risk
}

summary_df = pd.DataFrame(list(final_summary.items()), columns=['Metric', 'Value'])
summary_df.to_csv('../reports/advanced_analytics_summary.csv', index=False)

print(f"\n📊 FINAL ANALYTICS SUMMARY:")
print(summary_df.to_string(index=False))
print(f"\n✅ All advanced analytics reports saved to ../reports/")